In [1]:
import argparse,logging, os, math
import mxnet as mx
from mxnet import image
from mxnet import nd, gluon, autograd, init
from mxnet.gluon.data.vision import ImageFolderDataset
from mxnet.gluon.data import DataLoader
from mxnet.gluon import nn
from tensorboardX import SummaryWriter
import numpy as np
import shutil
import _pickle as cPickle
from mxnet.gluon.parameter import Parameter, ParameterDict
import subprocess
import time
from mxnet.gluon.loss import Loss, _apply_weighting, _reshape_like

from IPython.core.debugger import Tracer

from resnext_w_d_maxmin_v2_128 import resnext

# import os
# os.environ['MXNET_GPU_MEM_POOL_TYPE'] = 'Round'

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(message)s')
console = logging.StreamHandler()
console.setFormatter(formatter)
logger.addHandler(console)

In [3]:
class Options:
    def __init__(self):
        self.gpus = '0,1,2,3,4,5,6,7' #the gpus will be used, e.g "0,1,2,3"
        self.data_dir = '/tanData/datasets/imagenet/data' #the input data directory
        self.log_dir = '/tanData/logs'
        self.model_dir ='/tanData/models'
        self.exp_name = 'semisup_exp1_128K'
        self.data_type = 'imagenet' #the dataset type
        self.depth = 50 #the depth of resnet
        self.batch_size = 56 #the batch size
        self.num_group = 64 #the number of convolution groups
        self.drop_out = 0.0 #the probability of an element to be zeroed
        self.alpha_max = 0.5
        self.alpha_min = 0.5
        self.alpha_drm = 0.5
        self.alpha_rpn = 1.0
        self.alpha_kl = 0.5
        self.alpha_mm = 0.5
        self.alpha_min = 0.5
        self.alpha_max = 1.0 - self.alpha_min
        
        self.list_dir = './' #the directory which contain the training list file
        self.lr = 0.1 #initialization learning rate
        self.mom = 0.9 #momentum for sgd
        self.bn_mom = 0.9 #momentum for batch normlization
        self.wd = 0.0001 #weight decay for sgd
        self.workspace = 512 #memory space size(MB) used in convolution, 
                            #if xpu memory is oom, then you can try smaller vale, such as --workspace 256 
        self.num_classes = 1000 #the class number of your task
        self.aug_level = 2 # level 1: use only random crop and random mirror, 
                           #level 2: add scale/aspect/hsv augmentation based on level 1, 
                           #level 3: add rotation/shear augmentation based on level 2 
        self.num_examples = 1152000 # the number of training examples
        self.kv_store = 'device' # the kvstore type'
        self.model_load_epoch = 0 # load the model on an epoch using the model-load-prefix
        self.frequent = 50 # frequency of logging
        self.memonger = False # true means using memonger to save momory, https://github.com/dmlc/mxnet-memonger
        self.retrain = False # true means continue training
        
args = Options()

In [4]:
hdlr = logging.FileHandler('./log/log-semi128Kresnext-{}-{}.log'.format(args.data_type, args.depth))
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logging.info(args)

2018-09-25 11:37:43,833 - <__main__.Options object at 0x7f03f0ef7908>


In [5]:
kv = mx.kvstore.create(args.kv_store)
ctx = mx.cpu() if args.gpus is None else [mx.gpu(int(i)) for i in args.gpus.split(',')]
batch_size = args.batch_size
batch_size *= max(1, len(ctx))
begin_epoch = args.model_load_epoch if args.model_load_epoch else 0
if not os.path.exists("./model"):
    os.mkdir("./model")
model_prefix = "semi128Kresnext_{}_{}_{}_{}".format(args.data_type, args.depth, kv.rank, args.exp_name)
# model_prefix = "model/se-resnext-{}-{}-{}".format(args.data_type, args.depth, kv.rank)
arg_params = None
aux_params = None
if args.retrain:
    _, arg_params, aux_params = mx.model.load_checkpoint(model_prefix, args.model_load_epoch)

In [6]:
train_data_sup = mx.io.ImageRecordIter(
    path_imgrec         = os.path.join(args.data_dir, "train.rec") if args.data_type == 'cifar10' else
                          os.path.join(args.data_dir, "train_256_q90.rec") if args.aug_level == 1
                          else os.path.join(args.data_dir, "train_sup_240.rec") ,
    label_width         = 1,
    data_name           = 'data',
    label_name          = 'softmax_label',
    data_shape          = (3, 32, 32) if args.data_type=="cifar10" else (3, 128, 128),
    batch_size          = batch_size // 2,
    pad                 = 4 if args.data_type == "cifar10" else 0,
    fill_value          = 127,  # only used when pad is valid
    rand_crop           = True,
    max_random_scale    = 1.0,  # 480 with imagnet, 32 with cifar10
    min_random_scale    = 1.0 if args.data_type == "cifar10" else 1.0 if args.aug_level == 1 else 0.533,  # 256.0/480.0=0.533, 256.0/384.0=0.667 256.0/256=1.0
    max_aspect_ratio    = 0 if args.data_type == "cifar10" else 0 if args.aug_level == 1 else 0.25, # 0.25
    random_h            = 0 if args.data_type == "cifar10" else 0 if args.aug_level == 1 else 36,  # 0.4*90
    random_s            = 0 if args.data_type == "cifar10" else 0 if args.aug_level == 1 else 50,  # 0.4*127
    random_l            = 0 if args.data_type == "cifar10" else 0 if args.aug_level == 1 else 50,  # 0.4*127
    max_rotate_angle    = 0 if args.aug_level <= 2 else 10,
    max_shear_ratio     = 0 if args.aug_level <= 2 else 0.0, #0.1 args.aug_level = 3
    rand_mirror         = True,
    shuffle             = True,
    num_parts           = kv.num_workers,
    part_index          = kv.rank)

train_data_unsup = mx.io.ImageRecordIter(
    path_imgrec         = os.path.join(args.data_dir, "train.rec") if args.data_type == 'cifar10' else
                          os.path.join(args.data_dir, "train_256_q90.rec") if args.aug_level == 1
                          else os.path.join(args.data_dir, "train_unsup_240.rec") ,
    label_width         = 1,
    data_name           = 'data',
    label_name          = 'softmax_label',
    data_shape          = (3, 32, 32) if args.data_type=="cifar10" else (3, 128, 128),
    batch_size          = batch_size // 2,
    pad                 = 4 if args.data_type == "cifar10" else 0,
    fill_value          = 127,  # only used when pad is valid
    rand_crop           = True,
    max_random_scale    = 1.0,  # 480 with imagnet, 32 with cifar10
    min_random_scale    = 1.0 if args.data_type == "cifar10" else 1.0 if args.aug_level == 1 else 0.533,  # 256.0/480.0=0.533, 256.0/384.0=0.667 256.0/256=1.0
    max_aspect_ratio    = 0 if args.data_type == "cifar10" else 0 if args.aug_level == 1 else 0.25, # 0.25
    random_h            = 0 if args.data_type == "cifar10" else 0 if args.aug_level == 1 else 36,  # 0.4*90
    random_s            = 0 if args.data_type == "cifar10" else 0 if args.aug_level == 1 else 50,  # 0.4*127
    random_l            = 0 if args.data_type == "cifar10" else 0 if args.aug_level == 1 else 50,  # 0.4*127
    max_rotate_angle    = 0 if args.aug_level <= 2 else 10,
    max_shear_ratio     = 0 if args.aug_level <= 2 else 0.0, #0.1 args.aug_level = 3
    rand_mirror         = True,
    shuffle             = True,
    num_parts           = kv.num_workers,
    part_index          = kv.rank)

val_data = mx.io.ImageRecordIter(
    path_imgrec         = os.path.join(args.data_dir, "val.rec") if args.data_type == 'cifar10' else
                          os.path.join(args.data_dir, "val_128.rec"),
    label_width         = 1,
    data_name           = 'data',
    label_name          = 'softmax_label',
    batch_size          = batch_size,
    data_shape          = (3, 32, 32) if args.data_type=="cifar10" else (3, 128, 128),
    rand_crop           = False,
    rand_mirror         = False,
    num_parts           = kv.num_workers,
    part_index          = kv.rank)

In [7]:
class Normal(mx.init.Initializer):
    """Initializes weights with random values sampled from a normal distribution
    with a mean and standard deviation of `sigma`.
    """
    def __init__(self, mean=0, sigma=0.01):
        super(Normal, self).__init__(sigma=sigma)
        self.sigma = sigma
        self.mean = mean

    def _init_weight(self, _, arr):
        mx.random.normal(self.mean, self.sigma, out=arr)

In [8]:
def multi_factor_scheduler(begin_epoch, epoch_size, step=[30, 60, 90, 95, 110, 120], factor=0.1):
    step_ = [epoch_size * (x-begin_epoch) for x in step if x-begin_epoch > 0]
    return mx.lr_scheduler.MultiFactorScheduler(step=step_, factor=factor) if len(step_) else None

In [9]:
class L2Loss16bit(Loss):
    def __init__(self, weight=1., batch_axis=0, **kwargs):
        super(L2Loss16bit, self).__init__(weight, batch_axis, **kwargs)

    def hybrid_forward(self, F, pred, label, sample_weight=None):
        label = _reshape_like(F, label, pred)
        loss = F.square(pred - label)
        loss = _apply_weighting(F, loss, self._weight/2, sample_weight)
        return F.mean(F.mean(F.mean(loss, axis=3),axis=2), axis=1)
    
criterion = gluon.loss.SoftmaxCrossEntropyLoss()
L2_loss = gluon.loss.L2Loss()
L2_loss16 = L2Loss16bit()

acc_top1 = mx.metric.Accuracy()
acc_top5 = mx.metric.TopKAccuracy(5)
import datetime
writer = SummaryWriter(os.path.join(args.log_dir, args.exp_name))

In [10]:
def test(net, val_data, ctx):
    val_data.reset()
    
    acc_top1_val = mx.metric.Accuracy()
    acc_top5_val = mx.metric.TopKAccuracy(5)
    acc_top1_val_max = mx.metric.Accuracy()
    acc_top5_val_max = mx.metric.TopKAccuracy(5)
    acc_top1_val_min = mx.metric.Accuracy()
    acc_top5_val_min = mx.metric.TopKAccuracy(5)
    acc_top1_val.reset()
    acc_top5_val.reset()
    acc_top1_val_max.reset()
    acc_top5_val_max.reset()
    acc_top1_val_min.reset()
    acc_top5_val_min.reset()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        
        outputs = []
        outputsmax = []
        outputsmin = []
        for x, y in zip(data, label):
            cond = nd.zeros_like(y)
            zmax, zmin, _, _, _, _, _ = net(x, y, cond)
            z = args.alpha_max * zmax + args.alpha_min * zmin
            outputs.append(z)
            outputsmax.append(zmax)
            outputsmin.append(zmin)
            
            
        acc_top1_val.update(label, outputs)
        acc_top5_val.update(label, outputs)
        acc_top1_val_max.update(label, outputsmax)
        acc_top5_val_max.update(label, outputsmax)
        acc_top1_val_min.update(label, outputsmin)
        acc_top5_val_min.update(label, outputsmin)
        
        del data, label, z, zmax, zmin, x, y, cond

    _, top1 = acc_top1_val.get()
    _, top5 = acc_top5_val.get()
    _, top1max = acc_top1_val_max.get()
    _, top5max = acc_top5_val_max.get()
    _, top1min = acc_top1_val_min.get()
    _, top5min = acc_top5_val_min.get()
    return (top1, top5, top1max, top5max, top1min, top5min)

In [11]:
def train(net, train_data_sup, train_data_unsup, val_data, num_epochs, ctx):
    epoch_size = max(int(args.num_examples / (batch_size//2) / kv.num_workers), 1)
    # lr_sch = multi_factor_scheduler(begin_epoch, epoch_size, step=[15, 30], factor=0.1)
    # trainer = gluon.Trainer(net.collect_params(), 'nag', {'learning_rate':args.lr, 'momentum':args.mom, 'wd':args.wd, 'lr_scheduler': lr_sch, 'multi_precision': True})
    
    trainer = gluon.Trainer(
        net.collect_params(), 'adam', {'learning_rate': 0.001, 'wd': args.wd})
    prev_time = datetime.datetime.now()
    best_top1_val = 0.; best_top1_valmax = 0.; best_top1_valmin = 0.
    best_top5_val = 0.; best_top5_valmax = 0.; best_top5_valmin = 0.
    log_interval = 100
    
    for epoch in range(begin_epoch, num_epochs):
        train_data_sup.reset()
        train_data_unsup.reset()
        
        if epoch == 10 or epoch == 18:
            trainer.set_learning_rate(0.1 * trainer.learning_rate)
        
        tic = time.time()
        btic = time.time()
        acc_top1.reset()
        acc_top5.reset()
        train_loss = 0
        num_batch = 0
        
        for i, (batch_sup, batch_unsup) in enumerate(zip(train_data_sup, train_data_unsup)):
            bs = batch_unsup.data[0].shape[0]
            
            data_sup = gluon.utils.split_and_load(batch_sup.data[0], ctx_list=ctx, batch_axis=0)
            label_sup = gluon.utils.split_and_load(batch_sup.label[0], ctx_list=ctx, batch_axis=0)
            data_unsup = gluon.utils.split_and_load(batch_unsup.data[0], ctx_list=ctx, batch_axis=0)
            label_unsup = gluon.utils.split_and_load(batch_unsup.label[0], ctx_list=ctx, batch_axis=0)
            
            loss = []
            outputs = []
            
            with autograd.record():
                for x_sup, x_unsup, y_sup, y_unsup in zip(data_sup, data_unsup, label_sup, label_unsup):
                    cond_sup = nd.ones_like(y_sup)
                    zmax_sup, zmin_sup, xhatmax_sup, xhatmin_sup, loss_mmmax_sup, loss_mmmin_sup, xbn_sup = net(x_sup, y_sup, cond_sup)
                    loss_xent_sup = args.alpha_max * criterion(zmax_sup, y_sup) + args.alpha_min * criterion(zmin_sup, y_sup)
                    loss_drm_sup = args.alpha_max * L2_loss(xhatmax_sup, xbn_sup) + args.alpha_min * L2_loss(xhatmin_sup, xbn_sup)
                    softmax_scores_sup = args.alpha_max * nd.softmax(zmax_sup) + args.alpha_min * nd.softmax(zmin_sup)
                    loss_kl_sup = -nd.sum(nd.log(1000.0*softmax_scores_sup + 1e-8) * softmax_scores_sup, axis=1)
                    loss_mm_sup = args.alpha_max * loss_mmmax_sup + args.alpha_min * loss_mmmin_sup
                    loss_total_sup = loss_xent_sup + args.alpha_drm * loss_drm_sup + args.alpha_kl * loss_kl_sup + args.alpha_mm * loss_mm_sup
                    
                    cond_unsup = nd.zeros_like(y_unsup)
                    zmax_unsup, zmin_unsup, xhatmax_unsup, xhatmin_unsup, loss_mmmax_unsup, loss_mmmin_unsup, xbn_unsup = net(x_unsup, y_unsup, cond_unsup)
                    loss_drm_unsup = args.alpha_max * L2_loss(xhatmax_unsup, xbn_unsup) + args.alpha_min * L2_loss(xhatmin_unsup, xbn_unsup)
                    softmax_scores_unsup = args.alpha_max * nd.softmax(zmax_unsup) + args.alpha_min * nd.softmax(zmin_unsup)
                    loss_kl_unsup = -nd.sum(nd.log(1000.0*softmax_scores_unsup + 1e-8) * softmax_scores_unsup, axis=1)
                    loss_mm_unsup = args.alpha_max * loss_mmmax_unsup + args.alpha_min * loss_mmmin_unsup
                    loss_total_unsup = args.alpha_drm * loss_drm_unsup + args.alpha_kl * loss_kl_unsup + args.alpha_mm * loss_mm_unsup
                    
                    z_sup = args.alpha_max * zmax_sup + args.alpha_min * zmin_sup

                    loss.append(loss_total_sup + loss_total_unsup)
                    outputs.append(z_sup)
                    
            for l in loss:
                l.backward()
            
            trainer.step(bs)
            
            acc_top1.update(label_sup, outputs)
            acc_top5.update(label_sup, outputs)
            train_loss += sum([l.sum().asscalar() for l in loss])
            num_batch += 1
            if log_interval and not i % log_interval:
                _, top1 = acc_top1.get()
                _, top5 = acc_top5.get()
                logging.info('Epoch[%d] Batch [%d] Lr: %f Speed: %f samples/sec   loss=%f   top1-acc=%f     top5-acc=%f'%(
                          epoch, i, trainer.learning_rate, batch_size*log_interval/(time.time()-btic), train_loss/(num_batch * batch_size), top1, top5))
                btic = time.time()
            
            del data_sup, label_sup, data_unsup, label_unsup, loss, outputs, 
            del x_sup, x_unsup, y_sup, y_unsup, cond_sup, cond_unsup, xbn_sup, xbn_unsup
            del zmax_sup, zmin_sup, xhatmax_sup, xhatmin_sup, loss_mmmax_sup, loss_mmmin_sup
            del loss_xent_sup, loss_drm_sup, softmax_scores_sup, loss_kl_sup, loss_mm_sup, loss_total_sup, z_sup
            del zmax_unsup, zmin_unsup, xhatmax_unsup, xhatmin_unsup, loss_mmmax_unsup, loss_mmmin_unsup
            del loss_drm_unsup, softmax_scores_unsup, loss_kl_unsup, loss_mm_unsup, loss_total_unsup
        
        _, top1 = acc_top1.get()
        _, top5 = acc_top5.get()
        train_loss /= num_batch * batch_size
        writer.add_scalars('acc', {'train_top1': top1}, epoch)
        writer.add_scalars('acc', {'train_top5': top5}, epoch)
        
        top1_val, top5_val, top1_valmax, top5_valmax, top1_valmin, top5_valmin = test(net=net, val_data=val_data, ctx=ctx)
        
        if top1_val > best_top1_val:
            best_top1_val = top1_val
            net.collect_params().save('%s/%s_best_top1.params'%(args.model_dir, model_prefix))
        
        if top1_valmax > best_top1_valmax:
            best_top1_valmax = top1_valmax
            net.collect_params().save('%s/%s_best_top1_max.params'%(args.model_dir, model_prefix))
            
        if top1_valmin > best_top1_valmin:
            best_top1_valmin = top1_valmin
            net.collect_params().save('%s/%s_best_top1_min.params'%(args.model_dir, model_prefix))
        
        if top5_val > best_top5_val:
            best_top5_val = top5_val
            net.collect_params().save('%s/%s_best_top5.params'%(args.model_dir, model_prefix))
        
        if top5_valmax > best_top5_valmax:
            best_top5_valmax = top5_valmax
            net.collect_params().save('%s/%s_best_top5_max.params'%(args.model_dir, model_prefix))
        
        if top5_valmin > best_top5_valmin:
            best_top5_valmin = top5_valmin
            net.collect_params().save('%s/%s_best_top5_min.params'%(args.model_dir, model_prefix))
        
        logging.info('[Epoch %d] training: acc-top1=%f acc-top5=%f loss=%f lr=%f'%(epoch, top1, top5, train_loss, trainer.learning_rate))
        logging.info('[Epoch %d] time cost: %f'%(epoch, time.time()-tic))
        logging.info('[Epoch %d] validation: acc-top1=%f acc-top5=%f best-acc-top1=%f best-acc-top5=%f'%(epoch, top1_val, top5_val, best_top1_val, best_top5_val))
        logging.info('[Epoch %d] validation: acc-top1-max=%f acc-top5-max=%f best-acc-top1-max=%f best-acc-top5-max=%f'%(epoch, top1_valmax, top5_valmax, best_top1_valmax, best_top5_valmax))
        logging.info('[Epoch %d] validation: acc-top1-min=%f acc-top5-min=%f best-acc-top1-min=%f best-acc-top5-min=%f'%(epoch, top1_valmin, top5_valmin, best_top1_valmin, best_top5_valmin))
        
        writer.add_scalars('acc', {'valid_top1': top1_val}, epoch)
        writer.add_scalars('acc', {'valid_top5': top5_val}, epoch)
        writer.add_scalars('acc', {'valid_top1_max': top1_valmax}, epoch)
        writer.add_scalars('acc', {'valid_top5_max': top5_valmax}, epoch)
        writer.add_scalars('acc', {'valid_top1_min': top1_valmin}, epoch)
        writer.add_scalars('acc', {'valid_top5_min': top5_valmin}, epoch)
        
        net.collect_params().save('%s/%s_current.params'%(args.model_dir, model_prefix))
        if not epoch % 10:
            net.collect_params().save('%s/%s_epoch_%i.params'%(args.model_dir, model_prefix, epoch))
    
    return best_top1_val, best_top5_val

In [12]:
ratio_list = [0.25, 0.125, 0.0625, 0.03125]   # 1/4, 1/8, 1/16, 1/32
if args.depth == 18:
    units = [2, 2, 2, 2]
elif args.depth == 34:
    units = [3, 4, 6, 3]
elif args.depth == 50:
    units = [3, 4, 6, 3]
elif args.depth == 101:
    units = [3, 4, 23, 3]
elif args.depth == 152:
    units = [3, 8, 36, 3]
elif args.depth == 200:
    units = [3, 24, 36, 3]
elif args.depth == 269:
    units = [3, 30, 48, 8]
else:
    raise ValueError("no experiments done on detph {}, you can do it youself".format(args.depth))

num_epochs = 200 if args.data_type == "cifar10" else 40

In [13]:
def run_train(ctx): 
    model = resnext(units=units, num_stage=4, filter_list=[64, 256, 512, 1024, 2048] if args.depth >=50 else [64, 64, 128, 256, 512], ratio_list=ratio_list, num_class=args.num_classes, num_group=args.num_group, data_type="imagenet", drop_out=args.drop_out, bn_mom=args.bn_mom)
    for param in model.collect_params().values():
        if param.name.find('conv') != -1 or param.name.find('dense') != -1:
            if param.name.find('weight') != -1:
                param.initialize(init=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2), ctx=ctx)
            else:
                param.initialize(init=mx.init.Zero(), ctx=ctx)
        elif param.name.find('batchnorm') != -1:
            if param.name.find('gamma') != -1:
                param.initialize(init=Normal(mean=1, sigma=0.02), ctx=ctx)
            else:
                param.initialize(init=mx.init.Zero(), ctx=ctx)
        elif param.name.find('insnorm') != -1:
            if param.name.find('gamma') != -1:
                param.initialize(init=Normal(mean=1, sigma=0.02), ctx=ctx)
            else:
                param.initialize(init=mx.init.Zero(), ctx=ctx)
        elif param.name.find('biasadder') != -1:
            param.initialize(init=mx.init.Zero(), ctx=ctx)
        else:
            param.initialize(init=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2), ctx=ctx)
    
    # model.collect_params().load('/tanData/models/semi128Kresnext_imagenet_50_0_semisup_exp1_128K_current.params', ctx=ctx, allow_missing=True)
    model.hybridize()
        
    best_top1_val, best_top5_val = train(net=model, train_data_sup=train_data_sup, train_data_unsup=train_data_unsup, val_data=val_data, num_epochs=num_epochs, ctx=ctx)

In [ ]:
run_train(ctx=ctx)

2018-09-25 11:39:49,072 - Epoch[0] Batch [0] Lr: 0.001000 Speed: 440.625724 samples/sec   loss=26.252530   top1-acc=0.000000     top5-acc=0.000000
2018-09-25 11:44:39,123 - Epoch[0] Batch [100] Lr: 0.001000 Speed: 154.460530 samples/sec   loss=4.011204   top1-acc=0.028598     top5-acc=0.227237
2018-09-25 11:49:21,871 - Epoch[0] Batch [200] Lr: 0.001000 Speed: 158.448539 samples/sec   loss=3.117767   top1-acc=0.019412     top5-acc=0.144723
2018-09-25 11:54:04,140 - Epoch[0] Batch [300] Lr: 0.001000 Speed: 158.719476 samples/sec   loss=2.801885   top1-acc=0.016404     top5-acc=0.103954
2018-09-25 11:58:53,626 - Epoch[0] Batch [400] Lr: 0.001000 Speed: 154.764820 samples/sec   loss=2.614230   top1-acc=0.015464     top5-acc=0.085690
2018-09-25 12:03:45,604 - Epoch[0] Batch [500] Lr: 0.001000 Speed: 153.439314 samples/sec   loss=2.496557   top1-acc=0.013518     top5-acc=0.084207
2018-09-25 12:08:31,848 - Epoch[0] Batch [600] Lr: 0.001000 Speed: 156.518532 samples/sec   loss=2.398910   top1-